In [64]:
import ecdsa
import hashlib
import base58 
import binascii

from ecdsa import SigningKey

In [65]:
# SECP256k1 is the Bitcoin elliptic curve
prvk1 = SigningKey.generate(curve=ecdsa.SECP256k1) 
pubk1 = prvk1.get_verifying_key()

prvk2 = SigningKey.generate(curve=ecdsa.SECP256k1) 
pubk2 = prvk2.get_verifying_key()

dhk1 = prvk1.privkey.secret_multiplier * pubk2.pubkey.point
dhk2 = prvk2.privkey.secret_multiplier * pubk1.pubkey.point

m = hashlib.sha256()
m.update(dhk1.x().to_bytes(length=32, byteorder="big"))
m.update(dhk1.y().to_bytes(length=32, byteorder="big"))
shhex1 = m.hexdigest()

m = hashlib.sha256()
m.update(dhk2.x().to_bytes(length=32, byteorder="big"))
m.update(dhk2.y().to_bytes(length=32, byteorder="big"))
shhex2 = m.hexdigest()

shprvk = SigningKey.from_secret_exponent(int(shhex1, 16), curve=ecdsa.SECP256k1)

In [66]:
# Generate the bitcoin private key
raw_bytes = binascii.unhexlify('80') + shprvk.privkey.secret_multiplier.to_bytes(32, "big")
print(hex(shprvk.privkey.secret_multiplier))

addr = base58.b58encode_check(raw_bytes)
print(addr)

0xd9f4553c7f53aba8ea829e2baedc4b902e1a1edf18f9ad2a2915bd7eb469feab
5KUGwAsXMW9T7ZsjGLKEWP8EwU6J8jz3ohn7V9wwRhKLKTuKr7G


In [67]:
# Generate corresponding public key
shpubk = shprvk.get_verifying_key()

# Pack as 0x04 + x + y
raw_bytes = binascii.unhexlify('04') + shpubk.pubkey.point.x().to_bytes(32, "big") + shpubk.pubkey.point.y().to_bytes(32, "big")

# 0x00 + ripemp160(sha256(packed bytes))
hashed =  binascii.unhexlify('00') + hashlib.new("ripemd160", hashlib.sha256(raw_bytes).digest()).digest()

# base58 + checksum
addr = base58.b58encode_check(hashed)
print(addr)

17xSEuATPmzh7mBq6vY5Ti3DAR58Usefh2
